In [1]:
import json
import numpy as np
import pandas as pd
import os

from tensorflow.keras import Model
import tensorflow as tf

from tensorflow import keras
import zipfile
import pickle
from sklearn.utils import class_weight

from elmo_dataset_joiner import DatasetWorker, VocabularyWorker
from elmo_performance import PerformanceViewer
from model_factory import ModelFactory

np.random.seed(1234567890)
tf.random.set_seed(1234567890)       

In [2]:
def resetSeeds():
    np.random.seed(1234567890)
    tf.random.set_seed(1234567890)    

In [3]:
#tensorflow stuff
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [4]:
#tensorflow stuff in case mem overflows or something like that
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
## first step is to create the different datasets for sentiments, aspects and modifiers
extracts = ["sentiments", "aspects", "modifiers"]

vw= VocabularyWorker()

#data_laptop contains all reviews inclusive the test data
filename = r'data_laptop_absa.json'

#Embedding list from pickle file
embedding = pickle.load( open("elmo_embedding.pkl", "rb" ) )

##test_id_list contains all the test data which we keep seperate from the other data
test_id_list = pickle.load(open("test_id_list.pkl", "rb"))

with open(filename,'r', encoding='utf8') as infile:
    review_data = json.load(infile)

#confusion matrix will get stored inside here
# idx 0 -> sentiments, idx 1 -> aspects, idx 2 -> modifiers
results = []
filename = r'data_laptop_absa.json'

#gridsearch evaluated 5 epochs for sentiments and modifiers and 4 for aspects
epochs_parameter = {"sentiments":5, "modifiers":5, "aspects":4}

for extract in extracts:
    extraction_of = extract
    print("start building model for ", extraction_of)
    
    max_seq_length = 100
    ds = DatasetWorker(review_data)
    
    #tokenize data
    ds.applyPreprocessing()
    #todo set extraction_of correct
    ds.setExtractionOf(extraction_of)
    
    #options for splitDataset = every_review, every_review_without_uncertain
    #what we get is for example 
    #tokens[0]  [['computer', 'works', 'great', '.'], 1, 'train']
    #labels[0] ['O', 'O', 'O', 'O']
    tokens, labels =ds.splitDataset("every_review_without_uncertain", test_id_list)
    

    #build vocab and add embedding#
    #with elmo, every token of each sentence gets its individual elmo vecotr

    # we have a total of 54352 vectors
    vocab_size = len (embedding)

    #each vector has 1024 dimensions (smallest elmo dataset)
    embed_size = 1024

    embedding_vectors = np.zeros((vocab_size, embed_size))

    #embedding-vectors for Embedding layer 
    for i,embedding_element in enumerate(embedding):
        #we only extract the vector data
        token_vector = embedding_element[1]
        embedding_vectors[i] = token_vector
        
    #extract labelclasses
    all_labelclasses = set()
    for row in labels:
        all_labelclasses.update(row)
    all_labelclasses=list(all_labelclasses)
    all_labelclasses.sort()

    labelclass_to_id = dict(zip(all_labelclasses,list(range(len(all_labelclasses)))))
  
    
    #define number of labelclasses as n_tags
    n_tags = len(list(labelclass_to_id.keys()))

    #max sequence length defines the length each review should get padded to
    max_seq_length = 100

    #convert token and label to ids according to the embedding and split into train and test
    #test_tokens is needed for calculation of the confusion matrix
    #x_train,x_test, y_train, y_test are for training and testing respectively
    # train_tokens and test_tokens are used for better analyzing the data because they dont contain padded elements
    #because y data changes per extraction_of we have to let this run through every time
    x_train, x_test, y_train, y_test , train_tokens, test_tokens = vw.convert_tokens_labels_list_to_ids_list(tokens, labels, embedding, max_seq_length, labelclass_to_id, n_tags)
    
    ##get the specific model
    resetSeeds()
    model_factory = ModelFactory()
    model = model_factory.getModel(extraction_of, max_seq_length, vocab_size, embedding_vectors, n_tags)
    
    history = model.fit(
    x_train, y_train,
    batch_size=16,
    validation_split = 0.2,
    verbose = 1,
    epochs=epochs_parameter[extraction_of])
    
    
      #to evaluate metrics
    performance = PerformanceViewer(labelclass_to_id)
    
    results.append(performance.classicEvalCrf(model,x_test,y_test, test_tokens))
    

tokenize dataset: 3101it [00:00, 387745.55it/s]
split dataset labels: 3101it [00:00, 33354.88it/s]

start building model for  sentiments



Converting tokens & labels to ids : 100%|█████████████████████████████████████████| 8698/8698 [00:30<00:00, 283.40it/s]


generating model for sentiments
Epoch 1/5
326/326 [==============================] - 75s 230ms/step - crf_loss: 4.5431 - accuracy: 0.9845 - val_crf_loss_val: 1.9165 - val_val_accuracy: 0.9874
Epoch 2/5
326/326 [==============================] - 73s 224ms/step - crf_loss: 3.1162 - accuracy: 0.9888 - val_crf_loss_val: 1.7634 - val_val_accuracy: 0.9877
Epoch 3/5
326/326 [==============================] - 73s 224ms/step - crf_loss: 2.7972 - accuracy: 0.9893 - val_crf_loss_val: 1.7153 - val_val_accuracy: 0.9872
Epoch 4/5
326/326 [==============================] - 73s 224ms/step - crf_loss: 2.5243 - accuracy: 0.9895 - val_crf_loss_val: 1.7718 - val_val_accuracy: 0.9880
Epoch 5/5
326/326 [==============================] - 73s 224ms/step - crf_loss: 2.3138 - accuracy: 0.9900 - val_crf_loss_val: 1.7253 - val_val_accuracy: 0.9879


tokenize dataset: 3101it [00:00, 281960.08it/s]
split dataset labels: 3101it [00:00, 34088.24it/s]

...

generating model for aspects
Epoch 1/4
329/329 [==============================] - 77s 235ms/step - crf_loss: 4.7808 - accuracy: 0.9841 - val_crf_loss_val: 0.5044 - val_val_accuracy: 0.9881
Epoch 2/4
329/329 [==============================] - 76s 232ms/step - crf_loss: 3.4579 - accuracy: 0.9882 - val_crf_loss_val: 0.9092 - val_val_accuracy: 0.9889
Epoch 3/4
329/329 [==============================] - 76s 232ms/step - crf_loss: 3.0839 - accuracy: 0.9889 - val_crf_loss_val: 0.5085 - val_val_accuracy: 0.9888
Epoch 4/4
329/329 [==============================] - 76s 232ms/step - crf_loss: 2.7754 - accuracy: 0.9895 - val_crf_loss_val: 0.7512 - val_val_accuracy: 0.9884


tokenize dataset: 3101it [00:00, 387676.21it/s]
split dataset labels: 3101it [00:00, 33717.70it/s]


Precision 0.677
Recall 0.582
F1-measure 0.626
start building model for  modifiers



Converting tokens & labels to ids : 100%|█████████████████████████████████████████| 8830/8830 [00:29<00:00, 296.61it/s]


generating model for modifiers
Epoch 1/5
332/332 [==============================] - 75s 226ms/step - crf_loss: 2.4435 - accuracy: 0.9921 - val_crf_loss_val: 0.4910 - val_val_accuracy: 0.9958
Epoch 2/5
332/332 [==============================] - 74s 224ms/step - crf_loss: 1.3975 - accuracy: 0.9956 - val_crf_loss_val: 0.3517 - val_val_accuracy: 0.9959
Epoch 3/5
332/332 [==============================] - 74s 224ms/step - crf_loss: 1.2769 - accuracy: 0.9958 - val_crf_loss_val: 0.2410 - val_val_accuracy: 0.9955
Epoch 4/5
332/332 [==============================] - 75s 224ms/step - crf_loss: 1.1691 - accuracy: 0.9959 - val_crf_loss_val: 0.1894 - val_val_accuracy: 0.9960
Epoch 5/5
332/332 [==============================] - 74s 224ms/step - crf_loss: 1.0765 - accuracy: 0.9961 - val_crf_loss_val: 0.3442 - val_val_accuracy: 0.9960

Precision 0.817
Recall 0.508
F1-measure 0.627


In [6]:
# display the results
overall_f1 = 0
for result in results:
    display(result[0])
    print("precision: ","{:.3f}".format(result[1]))
    print("recall: ","{:.3f}".format(result[2]))
    print("f1: ","{:.3f}".format(result[3]))
    overall_f1 += result[3]
    
print("overall f1-score ", "{:.3f}".format((overall_f1)/3))

,B_S_pred,I_S_pred,O_pred
B_S_true,1284.0,5.0,1156.0
I_S_true,84.0,33.0,980.0
O_true,307.0,22.0,33939.0


precision:  0.752
recall:  0.515
f1:  0.612


,B_A_pred,I_A_pred,O_pred
B_A_true,1007.0,28.0,1038.0
I_A_true,48.0,299.0,694.0
O_true,604.0,305.0,33885.0


precision:  0.677
recall:  0.582
f1:  0.626


,BM_pred,IM_pred,O_pred
BM_true,382.0,0.0,541.0
IM_true,14.0,27.0,193.0
O_true,151.0,8.0,36838.0


precision:  0.817
recall:  0.508
f1:  0.627
overall f1-score  0.621
